### PART II : Using NLP Pipeline on one Arabic article

#### I - Scrap the article :

In [1]:
from bs4 import BeautifulSoup
import requests
import re


url = 'https://aljarida24.ma/p/actualites/271314/'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

para_div=soup.find('div',{'class':'content-text'})

header = soup.find('h1').text.strip()
header

'الاسكوا: الاحتلال وحالات الصراع عائق كبير أمام التنمية المستدامة في المنطقة العربية'

In [2]:
# Initialiser une variable pour stocker tous les paragraphes
all_paragraphs = ""

# Parcourir le contenu de para_div et concaténer les textes des paragraphes
for item in para_div.contents:
    if item.name == 'p':
        all_paragraphs += item.text.strip() + "  "

# Maintenant, all_paragraphs contient le texte de tous les paragraphes avec des espaces entre eux
print(all_paragraphs)

ذكرت اللجنة الاقتصادية والاجتماعية لغربي آسيا (الإسكوا)، أن الاحتلال وحالات الصراع وعدم الاستقرار في عدد من الدول العربية تشكل عائقا كبير جدا أمام تحقيق أهداف التنمية المستدامة في المنطقة العربية.  وأوضحت المسؤولة بمجموعة تنسيق العمل على خطة 2030 وأهـداف التنمية المستدامة في الإسكوا، جنى البابا، أن هذه العوامل تحول دون تحقيق الاهداف التنموية المرجوة في تسع بلدان عربية تضم حوالي 40 بالمائة من سكان المنطقة العربية، مع ما لذلك من تأثير على المنطقة بأسرها.  وقالت المسؤولة الأممية في تصريحات أوردتها صحف عربية اليوم الخميس إن هناك عدة عوامل تضع البلدان العربية التي تعاني من النزاعات "في حلقة مفرغة" لا تستطيع الخروج منها بسهولة، وتجعلها غير قادرة على استعادة مسار التنمية، الأمر الذي يؤثر على أجيال كثيرة وليس فقط على الجيل الحاضر.  وسجلت أن هناك قاسمين مشتركين أساسيين في مختلف حالات الدول التي تعاني من الصراعات في المنطقة أو من تداعياتها، هي قضية النزوح واللجوء، حيث يوجد في المنطقة حوالي 9 ملايين لاجئ يعيشون في ظروف صعبة، ويؤثرون كذلك على الفئات أو المجتمعات التي تستضيفهم، علاوة على 19 مليون ن

#### II - Establishment of NLP Pipeline (Text Cleaning, Tokenization, Stop words, Discretization, Normalization).

In [3]:
# Entity Cleaner: Unites entity tags and fixs misspellings 
def tags_cleaner(entity):
  entity = re.sub('\n','',entity) # Remove the newline (\n)
  if entity in ['B-LOC', 'B-MIS', 'B-ORG','B-PER','I-LOC','I-MIS','I-ORG','I-PER','O']:
    return entity
  elif entity in ['B-MIS0','B-MIS1', 'B-MIS2', 'B-MIS3', 'B-MIS-1','B-MIS-2', 'B-MIS1`', 'B-MISS1']:
    return 'B-MIS'
  elif entity in ['I-MIS0','I-MIS1', 'I-MIS2', 'I-MIS3']:
    return 'I-MIS'
  elif entity in ['B-ENGLISH', 'B-SPANISH', 'OO', 'IO']:
    return 'O'
  elif entity == 'I--ORG':
    return 'I-ORG'
  else:
    print('Error with entity:', entity)

In [4]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    # remove HTML TAG
    html = re.compile('[<،,()"".#*?>]')
    text = html.sub(r'',text)
    
    # Remove urls:
    url = re.compile('https?://\S+|www\.S+')
    text = url.sub(r'',text)
    
    # Remove email id:
    email = re.compile('[A-Za-z0-2]+@[\w]+.[\w]+')
    text = email.sub(r'',text)
    
    # Remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    # Remove longation "EX : سلاممممم = سلامم"
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    # remove any leading and trailing whitespace characters    
    text = text.strip()
    return text
    
print(all_paragraphs)
print("-----------------------------------------------------------")
para = clean_str(all_paragraphs)
print(para)

ذكرت اللجنة الاقتصادية والاجتماعية لغربي آسيا (الإسكوا)، أن الاحتلال وحالات الصراع وعدم الاستقرار في عدد من الدول العربية تشكل عائقا كبير جدا أمام تحقيق أهداف التنمية المستدامة في المنطقة العربية.  وأوضحت المسؤولة بمجموعة تنسيق العمل على خطة 2030 وأهـداف التنمية المستدامة في الإسكوا، جنى البابا، أن هذه العوامل تحول دون تحقيق الاهداف التنموية المرجوة في تسع بلدان عربية تضم حوالي 40 بالمائة من سكان المنطقة العربية، مع ما لذلك من تأثير على المنطقة بأسرها.  وقالت المسؤولة الأممية في تصريحات أوردتها صحف عربية اليوم الخميس إن هناك عدة عوامل تضع البلدان العربية التي تعاني من النزاعات "في حلقة مفرغة" لا تستطيع الخروج منها بسهولة، وتجعلها غير قادرة على استعادة مسار التنمية، الأمر الذي يؤثر على أجيال كثيرة وليس فقط على الجيل الحاضر.  وسجلت أن هناك قاسمين مشتركين أساسيين في مختلف حالات الدول التي تعاني من الصراعات في المنطقة أو من تداعياتها، هي قضية النزوح واللجوء، حيث يوجد في المنطقة حوالي 9 ملايين لاجئ يعيشون في ظروف صعبة، ويؤثرون كذلك على الفئات أو المجتمعات التي تستضيفهم، علاوة على 19 مليون ن

<>:10: SyntaxWarning: invalid escape sequence '\S'
<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\S'
<>:14: SyntaxWarning: invalid escape sequence '\w'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10344\790905078.py:10: SyntaxWarning: invalid escape sequence '\S'
  url = re.compile('https?://\S+|www\.S+')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10344\790905078.py:14: SyntaxWarning: invalid escape sequence '\w'
  email = re.compile('[A-Za-z0-2]+@[\w]+.[\w]+')


In [5]:
from nltk.tokenize import word_tokenize

def tokenize(para):
 # Tokenize the text
 tokens = word_tokenize(para)

 # Filter tokens to keep only non-numeric ones
 filtered_tokens = [token for token in tokens if not token.isdigit()]

 # Regular expression pattern to match words starting with 'و'
 pattern = r'^و\w*'
    
 # Tokenize the words starting with 'و'
 custom_tokens = []
 for token in filtered_tokens:
    if re.match(pattern, token):
        custom_tokens.append('و')
        if len(token) > 1:
            custom_tokens.append(token[1:])
    else:
        custom_tokens.append(token)
 return custom_tokens

custom_tokens=tokenize(para)


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def stop_words(custom_tokens):
 # Load Arabic stopwords
 stop_words = set(stopwords.words('arabic'))

 # Filter out stop words
 filtered_sentence = [token for token in custom_tokens if not token in stop_words]
 return filtered_sentence

filtered_sentence= stop_words(custom_tokens)
print("Original Tokens:")
print(custom_tokens)
print("\nFiltered Tokens:")
print(filtered_sentence)

Original Tokens:
['ذكرت', 'اللجنه', 'الاقتصاديه', 'و', 'الاجتماعيه', 'لغربي', 'اسيا', 'الاسكوا', 'ان', 'الاحتلال', 'و', 'حالات', 'الصراع', 'و', 'عدم', 'الاستقرار', 'في', 'عدد', 'من', 'الدول', 'العربيه', 'تشكل', 'عائقا', 'كبير', 'جدا', 'امام', 'تحقيق', 'اهداف', 'التنميه', 'المستدامه', 'في', 'المنطقه', 'العربيه', 'و', 'اوضحت', 'المسؤوله', 'بمجموعه', 'تنسيق', 'العمل', 'علي', 'خطه', 'و', 'اهداف', 'التنميه', 'المستدامه', 'في', 'الاسكوا', 'جني', 'البابا', 'ان', 'هذه', 'العوامل', 'تحول', 'دون', 'تحقيق', 'الاهداف', 'التنمويه', 'المرجوه', 'في', 'تسع', 'بلدان', 'عربيه', 'تضم', 'حوالي', 'بالمائه', 'من', 'سكان', 'المنطقه', 'العربيه', 'مع', 'ما', 'لذلك', 'من', 'تاثير', 'علي', 'المنطقه', 'باسرها', 'و', 'قالت', 'المسؤوله', 'الامميه', 'في', 'تصريحات', 'اوردتها', 'صحف', 'عربيه', 'اليوم', 'الخميس', 'ان', 'هناك', 'عده', 'عوامل', 'تضع', 'البلدان', 'العربيه', 'التي', 'تعاني', 'من', 'النزاعات', 'في', 'حلقه', 'مفرغه', 'لا', 'تستطيع', 'الخروج', 'منها', 'بسهوله', 'و', 'تجعلها', 'غير', 'قادره', 'علي', 'استعاده'

#### III - Using the Stemming and Lemmatization 

In [7]:
from nltk import ISRIStemmer

stemmer = ISRIStemmer()
stemmed_tokens = [stemmer.stem(token) for token in filtered_sentence]

In [8]:
import stanza
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Download Arabic resources for Stanza
stanza.download('ar')

# Initialize Stanza pipeline for Arabic
nlp = stanza.Pipeline('ar', processors='tokenize,pos,lemma')

# Join tokenized words into a single string
arabic_text = ' '.join(filtered_sentence)

# Initialize Stanza pipeline for Arabic
nlp = stanza.Pipeline('ar', processors='tokenize,pos,lemma')

# Process the text
doc = nlp(arabic_text)

# Extract lemmas
lemmas = [word.lemma for sent in doc.sentences for word in sent.words]

print("filtered_sentence:")
print(filtered_sentence)
print("\nLemmatized Tokens:")
print(lemmas)


C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-04-08 02:01:18 INFO: Downloaded file to C:\Users\lenovo\stanza_resources\resources.json
2024-04-08 02:01:18 INFO: Downloading default packages for language: ar (Arabic) ...
2024-04-08 02:01:19 INFO: File exists: C:\Users\lenovo\stanza_resources\ar\default.zip
2024-04-08 02:01:23 INFO: Finished downloading models and saved to C:\Users\lenovo\stanza_resources
2024-04-08 02:01:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMeth

filtered_sentence:
['ذكرت', 'اللجنه', 'الاقتصاديه', 'الاجتماعيه', 'لغربي', 'اسيا', 'الاسكوا', 'ان', 'الاحتلال', 'حالات', 'الصراع', 'عدم', 'الاستقرار', 'عدد', 'الدول', 'العربيه', 'تشكل', 'عائقا', 'كبير', 'جدا', 'امام', 'تحقيق', 'اهداف', 'التنميه', 'المستدامه', 'المنطقه', 'العربيه', 'اوضحت', 'المسؤوله', 'بمجموعه', 'تنسيق', 'العمل', 'علي', 'خطه', 'اهداف', 'التنميه', 'المستدامه', 'الاسكوا', 'جني', 'البابا', 'ان', 'العوامل', 'تحول', 'تحقيق', 'الاهداف', 'التنمويه', 'المرجوه', 'بلدان', 'عربيه', 'تضم', 'حوالي', 'بالمائه', 'سكان', 'المنطقه', 'العربيه', 'لذلك', 'تاثير', 'علي', 'المنطقه', 'باسرها', 'قالت', 'المسؤوله', 'الامميه', 'تصريحات', 'اوردتها', 'صحف', 'عربيه', 'اليوم', 'الخميس', 'ان', 'عده', 'عوامل', 'تضع', 'البلدان', 'العربيه', 'تعاني', 'النزاعات', 'حلقه', 'مفرغه', 'تستطيع', 'الخروج', 'بسهوله', 'تجعلها', 'قادره', 'علي', 'استعاده', 'مسار', 'التنميه', 'الامر', 'يؤثر', 'علي', 'اجيال', 'كثيره', 'فقط', 'علي', 'الجيل', 'الحاضر', 'سجلت', 'ان', 'قاسمين', 'مشتركين', 'اساسين', 'مختلف', 'حالات', 'الد

In [9]:
lemmas = ' '.join(lemmas)
lemmas=clean_str(lemmas)
lemmas=tokenize(lemmas)
lemmas= stop_words(custom_tokens)
lemmas1 = ' '.join(lemmas)

In [10]:
nltk.download('universal_tagset')
pos=nltk.pos_tag(lemmas,tagset='universal')
# Print the tagged text
print(pos)

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[('ذكرت', 'ADJ'), ('اللجنه', 'NOUN'), ('الاقتصاديه', 'NOUN'), ('الاجتماعيه', 'NOUN'), ('لغربي', 'NOUN'), ('اسيا', 'NOUN'), ('الاسكوا', 'NOUN'), ('ان', 'NOUN'), ('الاحتلال', 'NOUN'), ('حالات', 'NOUN'), ('الصراع', 'NOUN'), ('عدم', 'NOUN'), ('الاستقرار', 'NOUN'), ('عدد', 'NOUN'), ('الدول', 'NOUN'), ('العربيه', 'NOUN'), ('تشكل', 'NOUN'), ('عائقا', 'NOUN'), ('كبير', 'NOUN'), ('جدا', 'NOUN'), ('امام', 'NOUN'), ('تحقيق', 'NOUN'), ('اهداف', 'NOUN'), ('التنميه', 'NOUN'), ('المستدامه', 'NOUN'), ('المنطقه', 'NOUN'), ('العربيه', 'NOUN'), ('اوضحت', 'NOUN'), ('المسؤوله', 'NOUN'), ('بمجموعه', 'NOUN'), ('تنسيق', 'NOUN'), ('العمل', 'NOUN'), ('علي', 'NOUN'), ('خطه', 'NOUN'), ('اهداف', 'NOUN'), ('التنميه', 'NOUN'), ('المستدامه', 'NOUN'), ('الاسكوا', 'NOUN'), ('جني', 'NOUN'), ('البابا', 'NOUN'), ('ان', 'NOUN'), ('العوامل', 'NOUN'), ('تحول', 'NOUN'), ('تحقيق', 'NOUN'), ('الاهداف', 'NOUN'), ('التنمويه', 'NOUN'), ('المرجوه', 'NOUN'), ('بلدان', 'NOUN'), ('عربيه', 'NOUN'), ('تضم', 'NOUN'), ('حوالي', 'NOUN'), (

### IV - Comparetion of the tow mechanisms (Stemming and Lemmatization)

Stemming and lemmatization are text preprocessing techniques in natural language processing (NLP). Specifically, they reduce the inflected forms of words
across a text data set to one common root word or dictionary form, also known as a “lemma” in computational linguistics.
Stemming and lemmatization are particularly helpful in information retrieval systems like search engines where users may submit a query with one word
(for example, meditate) but expect results that use any inflected form of the word (for example, meditates, meditation, etc.). 
Although they are identical, the choice depends on the situation and objectives. In our situation, lemmatization is more effective.


### V- Parts of Speech technics based on both Rule based and Machine learning approaches.

In [11]:
# Download and load the Arabic model
stanza.download('ar')  # Download the Arabic model
nlp = stanza.Pipeline('ar')  # Load the Arabic model

# Process the text and obtain POS tags
doc = nlp(lemmas1)

# Extract POS tags
pos_tags = []
for sentence in doc.sentences:
    for word in sentence.words:
        pos_tags.append((word.text, word.upos))

# Print POS tags
print(pos_tags)

2024-04-08 02:01:35 INFO: Downloaded file to C:\Users\lenovo\stanza_resources\resources.json
2024-04-08 02:01:35 INFO: Downloading default packages for language: ar (Arabic) ...
2024-04-08 02:01:36 INFO: File exists: C:\Users\lenovo\stanza_resources\ar\default.zip
2024-04-08 02:01:40 INFO: Finished downloading models and saved to C:\Users\lenovo\stanza_resources
2024-04-08 02:01:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-08 02:01:41 INFO: Downloaded file to C:\Users\lenovo\stanza_resources\resources.json
2024-04-08 02:01:43 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-04-08 02

[('ذكرت', 'VERB'), ('اللجنه', 'NOUN'), ('الاقتصاديه', 'ADJ'), ('الاجتماعيه', 'ADJ'), ('ل', 'ADP'), ('غربي', 'ADJ'), ('اسيا', 'X'), ('الاسكوا', 'X'), ('ان', 'SCONJ'), ('الاحتلال', 'NOUN'), ('حالات', 'NOUN'), ('الصراع', 'NOUN'), ('عدم', 'NOUN'), ('الاستقرار', 'NOUN'), ('عدد', 'NOUN'), ('الدول', 'NOUN'), ('العربيه', 'ADJ'), ('تشكل', 'VERB'), ('عائقا', 'NOUN'), ('كبير', 'ADJ'), ('جدا', 'NOUN'), ('امام', 'ADP'), ('تحقيق', 'NOUN'), ('اهداف', 'NOUN'), ('التنميه', 'X'), ('المستدام', 'ADJ'), ('ه', 'PRON'), ('المنطقه', 'X'), ('العربيه', 'ADJ'), ('اوضحت', 'VERB'), ('المسؤوله', 'NOUN'), ('ب', 'ADP'), ('مجموع', 'ADJ'), ('ه', 'PRON'), ('تنسيق', 'NOUN'), ('العمل', 'NOUN'), ('علي', 'ADP'), ('خطه', 'NOUN'), ('اهداف', 'X'), ('التنميه', 'X'), ('المستدام', 'ADJ'), ('ه', 'PRON'), ('الاسكوا', 'X'), ('جني', 'X'), ('البابا', 'X'), ('ان', 'SCONJ'), ('العوامل', 'NOUN'), ('تحول', 'VERB'), ('تحقيق', 'NOUN'), ('الاهداف', 'NOUN'), ('التنمويه', 'X'), ('المرجوه', 'ADJ'), ('ب', 'ADP'), ('لدان', 'NOUN'), ('عربي', 'ADJ'

### VI - NER Methods.

In [12]:
from farasa.ner import FarasaNamedEntityRecognizer 

named_entity_recognizer = FarasaNamedEntityRecognizer()

NamedEntityRecognized = named_entity_recognizer.recognize(lemmas1)
print("Named entity recognized:\n", NamedEntityRecognized)

Named entity recognized:
 ذكرت/O اللجنه/B-ORG الاقتصاديه/I-ORG الاجتماعيه/I-ORG لغربي/I-ORG اسيا/I-ORG الاسكوا/O ان/O الاحتلال/O حالات/O الصراع/O عدم/O الاستقرار/O عدد/O الدول/O العربيه/O تشكل/O عائقا/O كبير/O جدا/O امام/O تحقيق/O اهداف/O التنميه/O المستدامه/O المنطقه/O العربيه/O اوضحت/O المسؤوله/O بمجموعه/O تنسيق/O العمل/O علي/O خطه/O اهداف/O التنميه/O المستدامه/O الاسكوا/O جني/O البابا/B-PERS ان/O العوامل/O تحول/O تحقيق/O الاهداف/O التنمويه/O المرجوه/O بلدان/O عربيه/O تضم/O حوالي/O بالمائه/O سكان/O المنطقه/B-LOC العربيه/I-LOC لذلك/O تاثير/O علي/O المنطقه/O باسرها/O قالت/O المسؤوله/O الامميه/O تصريحات/O اوردتها/O صحف/O عربيه/O اليوم/O الخميس/O ان/O عده/O عوامل/O تضع/O البلدان/O العربيه/O تعاني/O النزاعات/O حلقه/O مفرغه/O تستطيع/O الخروج/O بسهوله/O تجعلها/O قادره/O علي/O استعاده/O مسار/O التنميه/O الامر/O يؤثر/O علي/O اجيال/O كثيره/O فقط/O علي/O الجيل/O الحاضر/O سجلت/O ان/O قاسمين/O مشتركين/O اساسين/O مختلف/O حالات/O الدول/O تعاني/O الصراعات/O المنطقه/O او/O تداعياتها/O قضيه/O النزوح/O

### VII : Synthesis

#### This first lab was a great opportunity to get familiar with the difference libraries of Scraping and  techniques of storing data in the database in addition of NLP Pipeline that we have seen in during class from Text Cleaning and Tokenization to Parts of Speech and NER Methods by using them in Arabic paragraph which was a bit difficult but we get to know this technologies more
